In [9]:
import pandas as pd
pd.set_option('display.max_columns',None)

In [10]:
import sys
sys.path.append("..")  # 상위 디렉토리를 sys.path에 추가

import requests
import pandas as pd
from mySetting import Client_ID, Client_Secret

# 네이버 API 키
client_id = Client_ID  # 클라이언트 ID
client_secret = Client_Secret  # 클라이언트 Secret

def get_coordinates(address):
    url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode'
    headers = {
        'X-NCP-APIGW-API-KEY-ID': client_id,
        'X-NCP-APIGW-API-KEY': client_secret,
    }
    params = {
        'query': address,
    }
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    
    if data['addresses']:
        x = data['addresses'][0]['x']
        y = data['addresses'][0]['y']
        return (x, y)
    else:
        return (None, None)

In [11]:
apart1 = pd.read_excel('../apart_data/아파트_실거래가_2022_2023.xlsx', skiprows=12)
apart2 = pd.read_excel('../apart_data/아파트_실거래가_2023_2024.xlsx', skiprows=12)

/Users/hdk/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/hdk/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [12]:
apart = pd.concat([apart2, apart1])

In [13]:
apart

,NO,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),동,층,매수자,매도자,건축년도,도로명,해제사유발생일,거래유형,중개사소재지,등기일자,주택유형
0,1,경기도 평택시 비전동,881,881,0,신명나리,59.97,202408,20,"17,800",-,14,개인,개인,1995,동부공원로 40,-,중개거래,경기 평택시,-,아파트
1,2,경기도 평택시 용이동,738,738,0,e편한세상평택용이2단지,84.91,202408,20,"33,000",-,20,개인,개인,2017,신흥1로 43,-,중개거래,경기 평택시,-,아파트
2,3,경기도 평택시 비전동,산76-17,76,17,외기노조,58.86,202408,20,"11,500",-,2,개인,개인,1985,평택4로 86,-,직거래,-,-,아파트
3,4,경기도 평택시 소사동,697,697,0,평택효성해링턴플레이스2단지,72.65,202408,19,"34,700",-,9,개인,개인,2019,소사3로 22,-,중개거래,경기 평택시,-,아파트
4,5,경기도 평택시 비전동,879-5,879,5,비전현대,59.72,202408,19,"20,000",-,8,개인,개인,1994,동부공원로 15,-,직거래,-,-,아파트
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5938,5939,경기도 평택시 서정동,263-1,263,1,휴먼파크리움,22.28,202208,22,"10,000",-,12,-,-,2018,서두물로35번길 113,-,중개거래,경기 평택시,-,아파트
5939,5940,경기도 평택시 지산동,851-10,851,10,이안힐시티타워2차아파트,56.48,202208,22,"20,000",-,11,-,-,2017,지산로20번길 91,-,직거래,-,-,아파트
5940,5941,경기도 평택시 지산동,781-9,781,9,이한렉스빌플러스,27.04,202208,22,"11,000",-,5,-,-,2017,송탄로 385,-,직거래,-,-,아파트
5941,5942,경기도 평택시 지산동,1094,1094,0,건영(지산),53.58,202208,22,"16,500",-,8,-,-,1993,지산2로 50,-,중개거래,경기 평택시,-,아파트


In [23]:
apart['주소'] = apart['시군구'] + ' ' + apart['번지']

In [24]:
apt = apart.reset_index()[['단지명','주소','층','계약년월','전용면적(㎡)','건축년도','거래금액(만원)']]

In [25]:
apt['위도'] = pd.NA
apt['경도'] = pd.NA

for i, address in enumerate(apt['주소']):
    lng, lat = get_coordinates(address)
    apt.at[i, '위도'] = lat
    apt.at[i, '경도'] = lng

In [26]:
apt

,단지명,주소,층,계약년월,전용면적(㎡),건축년도,거래금액(만원),위도,경도
0,신명나리,경기도 평택시 비전동 881,14,202408,59.97,1995,"17,800",36.9931272,127.1194389
1,e편한세상평택용이2단지,경기도 평택시 용이동 738,20,202408,84.91,2017,"33,000",37.0031991,127.1465691
2,외기노조,경기도 평택시 비전동 산76-17,2,202408,58.86,1985,"11,500",36.9952024,127.1025151
3,평택효성해링턴플레이스2단지,경기도 평택시 소사동 697,9,202408,72.65,2019,"34,700",36.9874051,127.1296995
4,비전현대,경기도 평택시 비전동 879-5,8,202408,59.72,1994,"20,000",36.9937759,127.1161204
...,...,...,...,...,...,...,...,...,...
11816,휴먼파크리움,경기도 평택시 서정동 263-1,12,202208,22.28,2018,"10,000",37.0578242,127.0585990
11817,이안힐시티타워2차아파트,경기도 평택시 지산동 851-10,11,202208,56.48,2017,"20,000",37.0753235,127.0578123
11818,이한렉스빌플러스,경기도 평택시 지산동 781-9,5,202208,27.04,2017,"11,000",37.0778512,127.0573304
11819,건영(지산),경기도 평택시 지산동 1094,8,202208,53.58,1993,"16,500",37.0763664,127.0639244


In [28]:
apt.to_csv('../preprocessed/아파트거래정보/아파트.csv', index=False)